# Create Combined All Data - Fixed Quarter Dates

1. credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv  
2. combined_calls.csv  
3. combined_corrected_tabular_financial_statements_data.parquet  

In [286]:
# Whether this is a sample/debug run or not
DEBUG = False

In [287]:
# Packages
import pandas as pd

In [288]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv
cr_and_dates = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv')
cr_and_dates

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,Source,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,source,days_since_call,days_since_rating
0,AAPL,2014-07-01 00:00:00.000000,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-04-01 00:00:00.000000,web,69,35
1,AAPL,2014-10-01 00:00:00.000000,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-07-01 00:00:00.000000,web,71,127
2,AAPL,2015-01-01 00:00:00.000000,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-10-01 00:00:00.000000,web,73,219
3,AAPL,2015-04-01 00:00:00.000000,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,AA,2014-05-27,2015-05-28,2015,2015-01-01 00:00:00.000000,web,64,42
4,AAPL,2015-07-01 00:00:00.000000,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,AA,2015-05-28,2015-08-25,2015,2015-04-01 00:00:00.000000,web,65,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8008,ZTS,2015-10-01 00:00:00.000000,2015-08-04,BBB,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,BBB,2014-01-31,2015-11-03,2015,2015-07-01 00:00:00.000000,web,58,244
8009,ZTS,2016-01-01 00:00:00.000000,2015-11-03,BBB,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,BBB,2015-01-30,2016-01-22,2015,2015-10-01 00:00:00.000000,web,59,59
8010,ZTS,2016-04-01 00:00:00.000000,2016-02-16,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,2016,2016-01-01 00:00:00.000000,web,45,70
8011,ZTS,2016-07-01 00:00:00.000000,2016-05-04,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,2016,2016-04-01 00:00:00.000000,web,58,161


In [289]:
# Convert earnings_call_date to date format
cr_and_dates['earnings_call_date'] = pd.to_datetime(cr_and_dates['earnings_call_date']).dt.date

In [290]:
# drop source column (about to get it again via the next join)
cr_and_dates = cr_and_dates.drop(columns = ['source'])

In [291]:
# check for duplicates on ticker by fixed_quarter_date
print(cr_and_dates[cr_and_dates.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call, days_since_rating]
Index: []


In [292]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Calls\calls.csv
if DEBUG:
    calls = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.parquet', nrows = 10)
else:
    calls = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.parquet')
    #calls = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.csv')
calls

,symbol,quarter,year,earnings_call_date,content,earnings_call_datetime,source,web
0,BCE,4,2012,2013-02-07,operators from ourselves. Our business perform...,2013-02-07,web,True
1,BCE,3,2012,2012-11-01,"Operator: Good morning, ladies and gentlemen. ...",2012-11-01,web,True
2,BCE,2,2012,2012-08-08,"Operator: Good morning, ladies and gentlemen. ...",2012-08-08,web,True
3,BCE,4,2013,2014-02-06,"Operator: Good morning, ladies and gentlemen. ...",2014-02-06,web,True
4,BCE,3,2013,2013-11-07,"Operator: Good morning, ladies and gentlemen. ...",2013-11-07,web,True
...,...,...,...,...,...,...,...,...
18457,WOR,1,2017,2016-09-28,"Operator: Good afternoon, ladies and gentlemen...",2016-09-28,web,True
18458,XIN,4,2017,2018-02-09,"Operator: Good day, everyone, and welcome to t...",2018-02-09,web,True
18459,XIN,3,2017,2017-11-11,"Operator: Good day, everyone, and welcome to t...",2017-11-11,web,True
18460,XIN,2,2017,2017-08-11,"Operator: Good day, everyone. And welcome to t...",2017-08-11,web,True


In [293]:
# check for duplicates on company by date
display(calls[calls.duplicated(subset=['symbol', 'earnings_call_date'], keep=False)])
calls.drop_duplicates(subset=['symbol', 'earnings_call_date'], inplace = True)
display(calls[calls.duplicated(subset=['symbol', 'earnings_call_date'], keep=False)])
calls

,symbol,quarter,year,earnings_call_date,content,earnings_call_datetime,source,web


,symbol,quarter,year,earnings_call_date,content,earnings_call_datetime,source,web


,symbol,quarter,year,earnings_call_date,content,earnings_call_datetime,source,web
0,BCE,4,2012,2013-02-07,operators from ourselves. Our business perform...,2013-02-07,web,True
1,BCE,3,2012,2012-11-01,"Operator: Good morning, ladies and gentlemen. ...",2012-11-01,web,True
2,BCE,2,2012,2012-08-08,"Operator: Good morning, ladies and gentlemen. ...",2012-08-08,web,True
3,BCE,4,2013,2014-02-06,"Operator: Good morning, ladies and gentlemen. ...",2014-02-06,web,True
4,BCE,3,2013,2013-11-07,"Operator: Good morning, ladies and gentlemen. ...",2013-11-07,web,True
...,...,...,...,...,...,...,...,...
18457,WOR,1,2017,2016-09-28,"Operator: Good afternoon, ladies and gentlemen...",2016-09-28,web,True
18458,XIN,4,2017,2018-02-09,"Operator: Good day, everyone, and welcome to t...",2018-02-09,web,True
18459,XIN,3,2017,2017-11-11,"Operator: Good day, everyone, and welcome to t...",2017-11-11,web,True
18460,XIN,2,2017,2017-08-11,"Operator: Good day, everyone. And welcome to t...",2017-08-11,web,True


# Earnings Call - Credit Score Merge

In [294]:
display(calls.columns)
display(cr_and_dates.columns)

Index(['symbol', 'quarter', 'year', 'earnings_call_date', 'content',
       'earnings_call_datetime', 'source', 'web'],
      dtype='object')

Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'Source', 'Rating Rank AAA is 10',
       'Next Rating', 'Next Rating Date', 'Previous Rating',
       'Previous Rating Date', 'next_rating_date_or_end_of_data',
       'credit_rating_year', 'previous_fixed_quarter_date', 'days_since_call',
       'days_since_rating'],
      dtype='object')

In [295]:
# convert earnings_call_date to date part
#cr_and_dates['earnings_call_date'] = pd.to_datetime(cr_and_dates['earnings_call_date']).dt.date
# drop earnings_call_datetime and web (identical as earnings_call_date and source)
calls.drop(["earnings_call_datetime", "web"], axis = 1, inplace=True)
# rename days_since_call and source 
cr_and_dates.rename(columns = {'days_since_call':'days_since_call_on_fixed_quarter', 'Source':'CR_source'}, inplace=True)
# rename symbol, source, and content
calls.rename(columns = {'symbol': 'ticker', "source": "Calls_source", "content": "transcript"}, inplace=True)

In [296]:
# print remaining columns
print(calls.columns)
print(calls.shape)
print(cr_and_dates.columns)
print(cr_and_dates.shape)
# Print type of columns
print(calls.dtypes)
print(cr_and_dates.dtypes)
# Print head of ticker, earnings_call_date in both datasets
print(calls[['ticker', 'earnings_call_date']].head())
print(cr_and_dates[['ticker', 'earnings_call_date']].head())
# Print all items with ticker == 'AAPL' in both datasets
print('aapl in calls')
print(calls[calls['ticker'] == 'AAPL'][['ticker', 'earnings_call_date']])
print('aapl in cr_and_dates')
print(cr_and_dates[cr_and_dates['ticker'] == 'AAPL'][['ticker', 'earnings_call_date']])

Index(['ticker', 'quarter', 'year', 'earnings_call_date', 'transcript',
       'Calls_source'],
      dtype='object')
(18462, 6)
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter',
       'days_since_rating'],
      dtype='object')
(8013, 17)
ticker                object
quarter                int64
year                   int64
earnings_call_date    object
transcript            object
Calls_source          object
dtype: object
ticker                              object
fixed_quarter_date                  object
earnings_call_date                  object
Rating                              object
Rating Agency Name                  object
rating_date            

In [297]:
# Inner join cr_and_dates (on columns ticker, earnings_call_date) with calls (on columns company, date)
print(type(cr_and_dates['earnings_call_date'][0]))
# convert cr_and_dates earnings_call_date to string
cr_and_dates['earnings_call_date'] = cr_and_dates['earnings_call_date'].astype(str)
print(type(cr_and_dates['earnings_call_date'][0]))
print(type(calls['earnings_call_date'][0]))
merged = pd.merge(cr_and_dates, calls, on=['ticker', 'earnings_call_date'], how='inner')
print('length of merged data')
print(len(merged))

# Convert earnings_call_date to date format
merged['earnings_call_date'] = pd.to_datetime(merged['earnings_call_date']).dt.date

# Range of date, year, credit_rating_year
for variable in ['earnings_call_date', 'year', 'credit_rating_year']:
    print(variable)
    print(merged[variable].min())
    print(merged[variable].max())
    print()

# Free up memory from calls
# del calls
# del call_with_dropout
# del cr_and_dates
merged.head()

<class 'datetime.date'>
<class 'str'>
<class 'str'>
length of merged data
8007
earnings_call_date
2010-05-03
2016-09-30

year
2010
2017

credit_rating_year
2010
2016



,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,days_since_call_on_fixed_quarter,days_since_rating,quarter,year,transcript,Calls_source
0,AAPL,2014-07-01 00:00:00.000000,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-04-24,2015-02-18,2014,2014-04-01 00:00:00.000000,69,35,2,2014,"Operator: Good day, everyone, and welcome to t...",web
1,AAPL,2014-10-01 00:00:00.000000,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-04-24,2015-02-18,2014,2014-07-01 00:00:00.000000,71,127,3,2014,"Operator: Good day, everyone, and welcome to t...",web
2,AAPL,2015-01-01 00:00:00.000000,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-04-24,2015-02-18,2014,2014-10-01 00:00:00.000000,73,219,4,2014,"Operator: Good day, everyone, and welcome to t...",web
3,AAPL,2015-04-01 00:00:00.000000,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,2014-05-27,2015-05-28,2015,2015-01-01 00:00:00.000000,64,42,1,2015,Operator: Good day ladies and gentlemen and we...,web
4,AAPL,2015-07-01 00:00:00.000000,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,2015-05-28,2015-08-25,2015,2015-04-01 00:00:00.000000,65,29,2,2015,"Operator: Good day everyone, and welcome to th...",web


In [298]:
# Keep only the date 
merged["fixed_quarter_date"] = pd.to_datetime(merged["fixed_quarter_date"]).dt.date
merged["previous_fixed_quarter_date"] = pd.to_datetime(merged["previous_fixed_quarter_date"]).dt.date
merged["rating_date"] = pd.to_datetime(merged["rating_date"]).dt.date

In [299]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, days_since_rating, quarter, year, transcript, Calls_source]
Index: []

[0 rows x 21 columns]


In [300]:
# check values of source
print(merged['Calls_source'].value_counts())

Calls_source
web    8007
Name: count, dtype: int64


In [301]:
print(merged.shape)
print(merged.columns)

(8007, 21)
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter',
       'days_since_rating', 'quarter', 'year', 'transcript', 'Calls_source'],
      dtype='object')


In [302]:
print("number of fix_quarter_date = rating_date:", sum(merged.fixed_quarter_date == merged.rating_date))
print("number of fix_quarter_date after rating_date:", sum(merged.fixed_quarter_date < merged.rating_date))
print("number of earnings_call_date = rating_date:", sum(merged.earnings_call_date == merged.rating_date))
print("number of earnings_call_date after rating_date:", sum(merged.earnings_call_date < merged.rating_date))

number of fix_quarter_date = rating_date: 26
number of fix_quarter_date after rating_date: 0
number of earnings_call_date = rating_date: 47
number of earnings_call_date after rating_date: 1619


In [303]:
merged.days_since_call_on_fixed_quarter.describe()

count    8007.000000
mean       56.977395
std        14.826100
min         0.000000
25%        54.000000
50%        61.000000
75%        66.000000
max        91.000000
Name: days_since_call_on_fixed_quarter, dtype: float64

In [304]:
# Get columns as for_year and for_quarter
merged.rename(columns={"year": "for_year", "quarter": "for_quarter"}, inplace=True)

In [305]:
merged[merged.for_year != merged.credit_rating_year].head(10)

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,days_since_call_on_fixed_quarter,days_since_rating,for_quarter,for_year,transcript,Calls_source
7,AAPL,2016-04-01,2016-01-26,AA,Standard & Poor's Ratings Services,2015-08-25,Supplementary,9,AA,2016-05-20,...,2015-06-02,2016-05-20,2015,2016-01-01,66,220,1,2016,"Operator: Good day, ladies and gentlemen, and ...",web
13,ABB,2013-07-01,2013-04-24,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2013-04-01,68,410,1,2013,"Operator: Ladies and gentlemen, good morning o...",web
14,ABB,2013-10-01,2013-07-25,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2013-07-01,68,502,2,2013,"Operator: Ladies and gentlemen, good morning o...",web
15,ABB,2014-01-01,2013-10-24,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2013-10-01,69,594,3,2013,"Operator: Ladies and gentlemen, good morning o...",web
16,ABB,2014-04-01,2014-02-13,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2014-01-01,47,684,4,2013,operator and the maintenance leader wants he c...,web
17,ABB,2014-07-01,2014-04-29,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2014-04-01,63,775,1,2014,"Operator: Ladies and gentlemen, good morning, ...",web
18,ABB,2014-10-01,2014-07-23,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2014-07-01,70,867,2,2014,"Operator: Ladies and gentlemen, good morning o...",web
19,ABB,2015-01-01,2014-10-22,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2014-10-01,71,959,3,2014,"Operator: Ladies and gentlemen, good morning o...",web
20,ABB,2015-04-01,2015-02-05,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2015-01-01,55,1049,4,2014,Operator: First question is from Mr. Ben Uglow...,web
21,ABB,2015-07-01,2015-04-29,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2015-04-01,63,1140,1,2015,"Operator: Ladies and gentlemen, good morning o...",web


In [306]:
merged.columns

Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter',
       'days_since_rating', 'for_quarter', 'for_year', 'transcript',
       'Calls_source'],
      dtype='object')

### Merge Financial Statements Data to CR-Calls data

In [307]:
# Load tabular financial statement data
df = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Tabular_Fin\combined_corrected_tabular_financial_statements_data.parquet')

# Perform merge
merged = pd.merge(merged, df, on=['ticker', 'for_year', 'for_quarter'], how='inner')    
display(merged.head())

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,debtRatio,debtRatioAlt,debtEquityRatio,equityMultiplier,enterpriseValueMultiplier,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,operatingCashFlowToSales,freeCashFlowToOperatingCashFlow
0,AAPL,2014-07-01,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,0.937781,1.215864,25.398794,1.714018,727.199724,0.552725,0.492055,0.773643,0.296587,0.890235
1,AAPL,2014-10-01,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,0.937781,0.456498,0.839921,1.839921,727.199724,0.426394,0.325315,0.539572,0.273963,0.762945
2,AAPL,2015-01-01,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,0.937781,0.518860,1.078397,2.078397,727.199724,0.558238,0.395949,0.583264,0.314555,0.709283
3,AAPL,2015-04-01,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,0.937781,0.529092,1.123557,2.123557,727.199724,1.442817,1.303122,0.833379,0.452044,0.903179
4,AAPL,2015-07-01,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,0.167963,0.506091,1.024666,2.024666,727.199724,0.823337,0.716499,0.625194,0.328926,0.870237


In [308]:
# Convert filingDate to date format
merged['filingDate'] = pd.to_datetime(merged['filingDate']).dt.date

In [309]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, days_since_rating, for_quarter, for_year, transcript, Calls_source, reportedCurrency, acceptedDate_balance_sheet, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory_balance_sheet, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabi

## Changes in Financial Variables

In [310]:
# Variables to difference
vars_to_diff = ['Altman_Z', 'Ratio_A', 'Ratio_B', 'Ratio_C', 'Ratio_D', 'Ratio_E', 'grossProfitRatio', 'ebitdaratio', 'operatingIncomeRatio', 'incomeBeforeTaxRatio', 'netIncomeRatio']

# Version of the data that is company, fixed quarter date, these columns
previous_quarter_data = (merged[['ticker', 'fixed_quarter_date'] + vars_to_diff].rename(columns = {var: var + '_previous' for var in vars_to_diff})
                                                                                .rename(columns = {'fixed_quarter_date': 'previous_fixed_quarter_date'})
                        )

# Join the previous quarter data
merged = pd.merge(merged, previous_quarter_data, on=['ticker', 'previous_fixed_quarter_date'], how='left')

# Check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

# Create differenced variables
for var in vars_to_diff:
    merged[var + '_diff'] = merged[var] - merged[var + '_previous']

# Print some examples
print(merged[['ticker', 'fixed_quarter_date', 'previous_fixed_quarter_date', 'Altman_Z', 'Altman_Z_previous', 'Altman_Z_diff']].head())

# Delete '_previous' columns
merged.drop(columns = [var + '_previous' for var in vars_to_diff], inplace = True)

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, days_since_rating, for_quarter, for_year, transcript, Calls_source, reportedCurrency, acceptedDate_balance_sheet, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory_balance_sheet, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabi

### Drops based on date disagreements

In [311]:
# Drop if filingDate is not in the fixed quarter
print('num cases where filingDate not in fixed quarter', len(merged[(merged['filingDate'] < merged['previous_fixed_quarter_date']) | (merged['filingDate'] > merged['fixed_quarter_date'])]))
merged = merged[(merged['filingDate'] >= merged['previous_fixed_quarter_date']) & (merged['filingDate'] <= merged['fixed_quarter_date'])]
# Drop if filingDate more than 45 days after earnings_call_date
print('num cases where filingDate more than 45 days after earnings_call_date', len(merged[(merged['filingDate'] - merged['earnings_call_date']) > pd.Timedelta('45 days 00:00:00')]))
merged = merged[(merged['filingDate'] - merged['earnings_call_date']) <= pd.Timedelta('45 days 00:00:00')]

num cases where filingDate not in fixed quarter 658
num cases where filingDate more than 45 days after earnings_call_date 3


### New Columns: Rating Changes

In [312]:
# Variable for rating on previous fixed_quarter_date
# Sort by ticker, fixed_quarter_date
merged = merged.sort_values(by=['ticker', 'fixed_quarter_date'])
# Shift the rating by 1
merged['rating_on_previous_fixed_quarter_date'] = merged.groupby('ticker')['Rating'].shift(1)
merged[['ticker', 'fixed_quarter_date', 'Rating', 'rating_on_previous_fixed_quarter_date']].head(10)

,ticker,fixed_quarter_date,Rating,rating_on_previous_fixed_quarter_date
0,AAPL,2014-07-01,AA,NaN
1,AAPL,2014-10-01,AA,AA
2,AAPL,2015-01-01,AA,AA
3,AAPL,2015-04-01,AA,AA
4,AAPL,2015-07-01,AA,AA
5,AAPL,2015-10-01,AA,AA
6,AAPL,2016-01-01,AA,AA
7,AAPL,2016-04-01,AA,AA
8,AAPL,2016-07-01,AA,AA
9,AAPL,2016-10-01,AA,AA


In [313]:
# Rating change variables
# Get numeric values for Rating and rating_on_next_earnings_call_date
# Encode Ratings as ordinal
rating_order = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'CC', 'C', 'D']
# Reverse
rating_order = rating_order[::-1]
# Create dictionary mapping to number values
rating_dict = {rating: i for i, rating in enumerate(rating_order, 1)}
# Create new column "Rating Rank AAA is 10"
merged['Rating Rank AAA is 10'] = merged['Rating'].map(rating_dict)
# Flag for investment grade - Rating Rank >= 7.0
merged['Investment_Grade'] = merged['Rating Rank AAA is 10'] >= 7.0
# Create new column "rating_on_previous_fixed_quarter_date AAA is 10"
merged['rating_on_previous_fixed_quarter_date AAA is 10'] = merged['rating_on_previous_fixed_quarter_date'].map(rating_dict)

# Cross tab Rating and 'Rating Rank AAA is 10'
display(pd.crosstab(merged['Rating'], merged['Rating Rank AAA is 10']))
# Cross tab rating_on_previous_fixed_quarter_date and 'rating_on_previous_fixed_quarter_date AAA is 10'
display(pd.crosstab(merged['rating_on_previous_fixed_quarter_date'], merged['rating_on_previous_fixed_quarter_date AAA is 10']))

Rating Rank AAA is 10,1,2,3,4,5,6,7,8,9,10
Rating,,,,,,,,,,
A,0,0,0,0,0,0,0,1209,0,0
AA,0,0,0,0,0,0,0,0,255,0
AAA,0,0,0,0,0,0,0,0,0,128
B,0,0,0,0,1061,0,0,0,0,0
BB,0,0,0,0,0,1756,0,0,0,0
BBB,0,0,0,0,0,0,2070,0,0,0
C,0,16,0,0,0,0,0,0,0,0
CC,0,0,14,0,0,0,0,0,0,0
CCC,0,0,0,165,0,0,0,0,0,0


rating_on_previous_fixed_quarter_date AAA is 10,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
rating_on_previous_fixed_quarter_date,,,,,,,,,,
A,0,0,0,0,0,0,0,1129,0,0
AA,0,0,0,0,0,0,0,0,230,0
AAA,0,0,0,0,0,0,0,0,0,120
B,0,0,0,0,971,0,0,0,0,0
BB,0,0,0,0,0,1614,0,0,0,0
BBB,0,0,0,0,0,0,1926,0,0,0
C,0,15,0,0,0,0,0,0,0,0
CC,0,0,13,0,0,0,0,0,0,0
CCC,0,0,0,154,0,0,0,0,0,0


In [314]:
# Variable for change before next call
def change(row):
    if row['Rating Rank AAA is 10'] < row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Downgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] > row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Upgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] == row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Same As Last Fixed Quarter Date'
    else:
        return None

# Apply function to create new column
merged['Change Direction Since Last Fixed Quarter Date'] = merged.apply(change, axis = 1)

# Also numeric version 'Change Since Last Fixed Quarter Date'
merged['Change Since Last Fixed Quarter Date'] = merged['Rating Rank AAA is 10'] - merged['rating_on_previous_fixed_quarter_date AAA is 10']

# Cross tab 'Change Direction Since Last Fixed Quarter Date' and 'Change Since Last Fixed Quarter Date'
display(pd.crosstab(merged['Change Direction Since Last Fixed Quarter Date'], merged['Change Since Last Fixed Quarter Date']))

Change Since Last Fixed Quarter Date,-2.0,-1.0,0.0,1.0,2.0
Change Direction Since Last Fixed Quarter Date,,,,,
Downgrade Since Last Fixed Quarter Date,16,127,0,0,0
Same As Last Fixed Quarter Date,0,0,5854,0,0
Upgrade Since Last Fixed Quarter Date,0,0,0,165,18


### Sector and Other Textual Company Information

From Kaggle: https://www.kaggle.com/datasets/aramacus/usa-public-companies

Supplemented with manually looked up sector data for missing sectors

In [315]:
# Load CSV 'C:\Users\ijyli\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv'
combined_sector_data = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv')
combined_sector_data.head()

,ticker,Sector
0,CTVA,Materials
1,ALCO,Consumer Staples
2,LMNR,Consumer Staples
3,SANW,Consumer Staples
4,TRC,Real Estate


In [316]:
combined_sector_data.shape

(3389, 2)

In [317]:
# Left join us_stocks with merged
merged = pd.merge(merged, combined_sector_data, on='ticker', how='left')
merged

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,ebitdaratio_diff,operatingIncomeRatio_diff,incomeBeforeTaxRatio_diff,netIncomeRatio_diff,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector
0,AAPL,2014-07-01,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,NaN,NaN,NaN,NaN,NaN,True,NaN,None,NaN,Information Technology
1,AAPL,2014-10-01,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,-0.012459,-0.023107,-0.022640,-0.016974,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
2,AAPL,2015-01-01,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,-0.014871,-0.009628,-0.007736,-0.005982,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
3,AAPL,2015-04-01,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,0.047734,0.059961,0.054951,0.040605,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
4,AAPL,2015-07-01,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,-0.001718,-0.009934,-0.007283,-0.007704,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6678,ZTS,2015-10-01,2015-08-04,BBB,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,...,-0.007788,-0.005240,-0.231690,-0.181217,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care
6679,ZTS,2016-01-01,2015-11-03,BBB,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,...,0.030978,0.028038,0.247855,0.187173,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care
6680,ZTS,2016-04-01,2016-02-16,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,-0.143885,-0.078861,-0.170716,-0.138415,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care
6681,ZTS,2016-07-01,2016-05-04,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,0.214569,0.119659,0.231554,0.158291,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care


In [318]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, days_since_rating, for_quarter, for_year, transcript, Calls_source, reportedCurrency, acceptedDate_balance_sheet, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory_balance_sheet, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabi

In [319]:
merged.Sector.isnull().sum()

0

## Exporting Data

In [320]:
# Clean up variables

# print column names
for col in merged.columns:
    print(col)

ticker
fixed_quarter_date
earnings_call_date
Rating
Rating Agency Name
rating_date
CR_source
Rating Rank AAA is 10
Next Rating
Next Rating Date
Previous Rating
Previous Rating Date
next_rating_date_or_end_of_data
credit_rating_year
previous_fixed_quarter_date
days_since_call_on_fixed_quarter
days_since_rating
for_quarter
for_year
transcript
Calls_source
reportedCurrency
acceptedDate_balance_sheet
cashAndCashEquivalents
shortTermInvestments
cashAndShortTermInvestments
netReceivables
inventory_balance_sheet
otherCurrentAssets
totalCurrentAssets
propertyPlantEquipmentNet
goodwill
intangibleAssets
goodwillAndIntangibleAssets
longTermInvestments
taxAssets
otherNonCurrentAssets
totalNonCurrentAssets
otherAssets
totalAssets
accountPayables
shortTermDebt
taxPayables
deferredRevenue
otherCurrentLiabilities
totalCurrentLiabilities
longTermDebt
deferredRevenueNonCurrent
deferredTaxLiabilitiesNonCurrent
otherNonCurrentLiabilities
totalNonCurrentLiabilities
otherLiabilities
capitalLeaseObligations


In [321]:
# delete columns Rating Agency Name, CR_Source, Calls_Source
merged.drop(columns=['Rating Agency Name', 'CR_source', 'Calls_source'], inplace=True)

In [322]:
# remove some drop_out
mask = merged.ticker.isin(["DRI", "MU", "XIN", "DEO"])

merged = merged[~mask]

### Removing BTU after Mar 3, 2015

In [323]:
merged.drop([789, 790, 791], inplace=True)
merged[df.ticker == "BTU"]

C:\Users\ijyli\AppData\Local\Temp\ipykernel_24572\199612304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.drop([789, 790, 791], inplace=True)
C:\Users\ijyli\AppData\Local\Temp\ipykernel_24572\199612304.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merged[df.ticker == "BTU"]


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,ebitdaratio_diff,operatingIncomeRatio_diff,incomeBeforeTaxRatio_diff,netIncomeRatio_diff,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector
3868,MPLX,2016-01-01,2015-10-29,BBB,2015-12-28,7,BBB,2016-01-29,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,None,NaN,Energy
3869,MPLX,2016-04-01,2016-02-03,BBB,2016-01-29,7,NaN,NaN,BBB,2015-12-28,...,-0.178638,-0.139727,-0.219553,-0.224496,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Energy
3870,MRC,2013-04-01,2013-02-22,B,2012-10-24,5,BB,2013-04-25,B,2012-10-05,...,NaN,NaN,NaN,NaN,NaN,False,NaN,None,NaN,Industrials
3871,MRC,2013-07-01,2013-05-03,BB,2013-04-25,6,BB,2014-04-17,B,2012-10-24,...,-0.010868,-0.013895,0.062577,0.040316,B,False,5.0,Upgrade Since Last Fixed Quarter Date,1.0,Industrials
3872,MRC,2013-10-01,2013-08-02,BB,2013-04-25,6,BB,2014-04-17,B,2012-10-24,...,-0.005176,0.005204,-0.004791,-0.000797,BB,False,6.0,Same As Last Fixed Quarter Date,0.0,Industrials
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930,MSCI,2015-04-01,2015-02-05,BB,2015-03-31,6,BB,2015-08-10,BB,2014-10-31,...,0.009728,0.004953,-0.036205,-0.028911,BB,False,6.0,Same As Last Fixed Quarter Date,0.0,Financials
3931,MSCI,2015-07-01,2015-04-30,BB,2015-03-31,6,BB,2015-08-10,BB,2014-10-31,...,-0.011281,-0.001159,0.013878,-0.009790,BB,False,6.0,Same As Last Fixed Quarter Date,0.0,Financials
3932,MSCI,2015-10-01,2015-07-31,BB,2015-08-10,6,BB,2016-03-07,BB,2015-03-31,...,0.027401,0.026355,0.027524,0.040237,BB,False,6.0,Same As Last Fixed Quarter Date,0.0,Financials
3933,MSCI,2016-01-01,2015-10-29,BB,2015-08-10,6,BB,2016-03-07,BB,2015-03-31,...,0.069081,0.041856,0.045431,0.032576,BB,False,6.0,Same As Last Fixed Quarter Date,0.0,Financials


### Changes in Financial Variables

In [324]:
# Variables to difference
vars_to_diff = ['Altman_Z', 'Ratio_A', 'Ratio_B', 'Ratio_C', 'Ratio_D', 'Ratio_E', 'grossProfitRatio', 'ebitdaratio', 'operatingIncomeRatio', 'incomeBeforeTaxRatio', 'netIncomeRatio'] + ['currentRatio', 'quickRatio', 'cashRatio', 'returnOnAssets', 'returnOnEquity', 'returnOnCapitalEmployed', 'EBITtoRevenue', 'debtRatio', 'debtRatioAlt', 'debtEquityRatio', 'equityMultiplier', 'enterpriseValueMultiplier', 'operatingCashFlowPerShare', 'freeCashFlowPerShare', 'cashPerShare', 'operatingCashFlowToSales', 'freeCashFlowToOperatingCashFlow']

# Version of the data that is company, fixed quarter date, these columns
previous_quarter_data = (merged[['ticker', 'fixed_quarter_date'] + vars_to_diff].rename(columns = {var: var + '_previous' for var in vars_to_diff})
                                                                                .rename(columns = {'fixed_quarter_date': 'previous_fixed_quarter_date'})
                        )

# # Join the previous quarter data
new_merged = pd.merge(merged, previous_quarter_data, on=['ticker', 'previous_fixed_quarter_date'], how='left')

# # Check for duplicates on ticker by fixed_quarter_date
print(new_merged[new_merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

# # Create differenced variables
for var in vars_to_diff:
    new_merged[var + '_diff'] = new_merged[var] - new_merged[var + '_previous']

# # Print some examples
display(new_merged[['ticker', 'fixed_quarter_date', 'previous_fixed_quarter_date', 'Altman_Z', 'Altman_Z_previous', 'Altman_Z_diff']].head())

# # Delete '_previous' columns
new_merged.drop(columns = [var + '_previous' for var in vars_to_diff], inplace = True)

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, rating_date, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, days_since_rating, for_quarter, for_year, transcript, reportedCurrency, acceptedDate_balance_sheet, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory_balance_sheet, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabilities, totalNonCurrentLiabilities, otherLiab

,ticker,fixed_quarter_date,previous_fixed_quarter_date,Altman_Z,Altman_Z_previous,Altman_Z_diff
0,AAPL,2014-07-01,2014-04-01,4.551851,NaN,NaN
1,AAPL,2014-10-01,2014-07-01,4.324703,4.551851,-0.227148
2,AAPL,2015-01-01,2014-10-01,5.727053,4.324703,1.402350
3,AAPL,2015-04-01,2015-01-01,3.992323,5.727053,-1.734730
4,AAPL,2015-07-01,2015-04-01,5.869492,3.992323,1.877169


In [325]:
new_merged.shape

(6672, 187)

## Export

In [326]:
# Save a parquet file
new_merged.to_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates.parquet', index=False)
new_merged

,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,debtRatio_diff,debtRatioAlt_diff,debtEquityRatio_diff,equityMultiplier_diff,enterpriseValueMultiplier_diff,operatingCashFlowPerShare_diff,freeCashFlowPerShare_diff,cashPerShare_diff,operatingCashFlowToSales_diff,freeCashFlowToOperatingCashFlow_diff
0,AAPL,2014-07-01,2014-04-23,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.000000,-0.759365,-24.558873,0.125902,0.000000,-0.126331,-0.166740,-0.234071,-0.022623,-0.127290
2,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.000000,0.062362,0.238477,0.238477,0.000000,0.131845,0.070634,0.043692,0.040592,-0.053662
3,AAPL,2015-04-01,2015-01-27,AA,2015-02-18,9,AA,2015-05-28,AA,2014-05-27,...,0.000000,0.010232,0.045159,0.045159,0.000000,0.884579,0.907173,0.250114,0.137489,0.193896
4,AAPL,2015-07-01,2015-04-27,AA,2015-06-02,9,AA,2015-08-25,AA,2015-05-28,...,-0.769818,-0.023001,-0.098891,-0.098891,0.000000,-0.619480,-0.586623,-0.208184,-0.123118,-0.032942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6667,ZTS,2015-10-01,2015-08-04,BBB,2015-01-30,7,BBB,2015-11-03,BBB,2014-01-31,...,-0.014131,0.020778,0.578263,0.579245,1.096292,0.124190,0.122016,-0.027820,0.049383,0.372951
6668,ZTS,2016-01-01,2015-11-03,BBB,2015-11-03,7,BBB,2016-01-22,BBB,2015-01-30,...,-0.009219,-0.005380,-0.168582,-0.170349,-23.366204,0.164706,0.152516,0.098181,0.064210,0.122147
6669,ZTS,2016-04-01,2016-02-16,BBB,2016-01-22,7,BBB,2016-12-23,BBB,2015-11-03,...,0.071666,0.049682,1.967787,1.968981,95.485237,0.149307,0.090904,1.130204,0.050171,-0.036465
6670,ZTS,2016-07-01,2016-05-04,BBB,2016-01-22,7,BBB,2016-12-23,BBB,2015-11-03,...,-0.015700,-0.021390,-1.009100,-1.011751,-103.258245,-0.455396,-0.383304,-0.958949,-0.174321,-0.590986


In [327]:
# Sample 100 rows to create all_data_sample.csv
if not DEBUG:
    all_data_sample = new_merged.sample(100)
    all_data_sample.to_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.csv', index=False)
    # also save to xlsx
    all_data_sample.to_excel(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.xlsx', index=False)

## Fragmented Version to Store on GitHub

In [328]:
new_merged.to_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates.parquet', index=False)
# Split dataset into pieces
num_pieces = 8
total_len_pieces = 0
# Delete previous pieces, all contents of '../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
import os
import shutil
folder = r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
# Save pieces
if not DEBUG:
    for i in range(num_pieces):
        # start index for piece rows
        start_index = i * len(new_merged) // num_pieces
        # end index for piece rows
        end_index = (i + 1) * len(new_merged) // num_pieces
        # get piece
        piece = new_merged[start_index:end_index]
        piece.to_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates/all_data_fixed_quarter_dates_piece_' + str(i) + '.parquet', index=False)
        display(piece.head(3))
        print(len(piece))
        total_len_pieces += len(piece)
    # save 100 row sample
    new_merged.sample(100).to_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates_Sample/all_data_fixed_quarter_dates_sample.parquet', index=False)


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,debtRatio_diff,debtRatioAlt_diff,debtEquityRatio_diff,equityMultiplier_diff,enterpriseValueMultiplier_diff,operatingCashFlowPerShare_diff,freeCashFlowPerShare_diff,cashPerShare_diff,operatingCashFlowToSales_diff,freeCashFlowToOperatingCashFlow_diff
0,AAPL,2014-07-01,2014-04-23,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.0,-0.759365,-24.558873,0.125902,0.0,-0.126331,-0.166740,-0.234071,-0.022623,-0.127290
2,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.0,0.062362,0.238477,0.238477,0.0,0.131845,0.070634,0.043692,0.040592,-0.053662


834


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,debtRatio_diff,debtRatioAlt_diff,debtEquityRatio_diff,equityMultiplier_diff,enterpriseValueMultiplier_diff,operatingCashFlowPerShare_diff,freeCashFlowPerShare_diff,cashPerShare_diff,operatingCashFlowToSales_diff,freeCashFlowToOperatingCashFlow_diff
834,BZH,2015-04-01,2015-01-30,CCC,2014-07-31,4,CCC,2015-07-27,CCC,2014-06-13,...,0.029283,0.005318,0.303344,0.303344,-327.236321,-5.869683,-7.651196,-7.025231,-0.384878,0.060135
835,BZH,2015-07-01,2015-04-30,CCC,2014-07-31,4,CCC,2015-07-27,CCC,2014-06-13,...,-0.016991,0.002421,0.146520,0.146520,468.851793,0.000000,1.656477,0.281739,0.046849,0.132547
836,BZH,2015-10-01,2015-08-04,CCC,2015-07-27,4,CCC,2016-03-11,CCC,2014-07-31,...,-0.011825,-0.003077,-0.185279,-0.185279,-105.108463,0.000000,-0.172621,-0.661583,0.019473,-0.028101


834


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,debtRatio_diff,debtRatioAlt_diff,debtEquityRatio_diff,equityMultiplier_diff,enterpriseValueMultiplier_diff,operatingCashFlowPerShare_diff,freeCashFlowPerShare_diff,cashPerShare_diff,operatingCashFlowToSales_diff,freeCashFlowToOperatingCashFlow_diff
1668,EAT,2016-07-01,2016-04-19,BBB,2014-05-28,7,BBB,2016-08-11,BBB,2013-08-02,...,0.056920,0.059226,3.467307,3.467307,-4.371133,0.682734,0.742070,0.013016,0.035135,0.092871
1669,EAT,2016-10-01,2016-08-11,BB,2016-09-13,6,NaN,NaN,BBB,2016-08-11,...,-0.032403,-0.018923,-0.799252,-0.799252,-4.051161,-0.832153,-1.069955,-0.652250,-0.066758,-0.219920
1670,EBAY,2015-07-01,2015-04-22,AA,2015-04-10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


...

834


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,debtRatio_diff,debtRatioAlt_diff,debtEquityRatio_diff,equityMultiplier_diff,enterpriseValueMultiplier_diff,operatingCashFlowPerShare_diff,freeCashFlowPerShare_diff,cashPerShare_diff,operatingCashFlowToSales_diff,freeCashFlowToOperatingCashFlow_diff
5004,PPG,2013-04-01,2013-01-14,BBB,2012-07-31,7,A,2013-06-20,BBB,2012-06-29,...,0.073747,-0.091038,-1.045852,-1.052295,12.152690,0.458876,0.118863,-0.008510,0.047697,-0.115095
5005,PPG,2013-07-01,2013-04-18,A,2013-06-20,8,A,2014-03-03,BBB,2012-07-31,...,-0.108934,-0.183734,-0.446911,0.222370,-0.082437,-2.731581,-2.353772,2.396703,-0.231763,0.977663
5006,PPG,2013-10-01,2013-07-18,A,2013-06-20,8,A,2014-03-03,BBB,2012-07-31,...,-0.010869,0.003939,0.000764,-0.025599,-8.776158,2.337361,2.246004,-2.582465,0.169087,-0.884711


834


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,debtRatio_diff,debtRatioAlt_diff,debtEquityRatio_diff,equityMultiplier_diff,enterpriseValueMultiplier_diff,operatingCashFlowPerShare_diff,freeCashFlowPerShare_diff,cashPerShare_diff,operatingCashFlowToSales_diff,freeCashFlowToOperatingCashFlow_diff
5838,TDS,2016-07-01,2016-05-06,BB,2014-11-24,6,NaN,NaN,BB,2014-04-04,...,0.000761,-0.002607,-0.010887,-0.009908,-0.780008,1.681308,2.446143,0.641852,0.148651,2.770444
5839,TDS,2016-10-01,2016-08-05,BB,2014-11-24,6,NaN,NaN,BB,2014-04-04,...,-0.001298,-0.000574,0.001679,0.006000,-0.539099,-0.844037,-0.504587,-1.422018,-0.077877,-0.145866
5840,TEN,2012-10-01,2012-07-26,B,2012-03-08,5,B,2013-03-28,B,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


834


In [329]:
# check total piece length and length of merged
print(total_len_pieces)
print(len(new_merged))

6672
6672
